In [88]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')

In [89]:
# загружаем датафрейм
df = pd.read_csv('../../data/raw/_data.csv')

In [90]:
# общие пропуски
df.isnull().sum()

Unnamed: 0                      0
ID  объявления                  0
Количество комнат            1041
Тип                             0
Метро                        1315
Адрес                           0
Площадь, м2                     0
Дом                             0
Парковка                    13417
Цена                            0
Телефоны                        0
Описание                        0
Ремонт                       2755
Площадь комнат, м2           8910
Балкон                       7978
Окна                         6613
Санузел                      2672
Можно с детьми/животными     6096
Дополнительно                 357
Название ЖК                 17520
Серия дома                  21205
Высота потолков, м          12162
Лифт                         5500
Мусоропровод                10522
Ссылка на объявление            0
dtype: int64

In [91]:
# для начала уберем все объявления не из москвы
def is_moscow(text):
    return text.split(',')[0] == 'Москва'

In [92]:
df = df[df['Адрес'].apply(is_moscow)]

In [93]:
# начинаем заполнять пропуски с "Ремонта"
df['Ремонт'].isnull().sum()

2463

In [94]:
# после долгих размышлений было принято решение заполнять по принципу моды в
# категориях квартир со схожей стоимостью. Может это того не стоит, но попробовать захотелось
# для этого сначала нужно достать цену аренды, тк это важный параметр

# решил пойти таким путём (в надежде на то, что все цены вбиты в одном формате)
def extract_price(text):
    dot_index = text.find('.')
    if dot_index != -1:
        return int(text[:dot_index])
    return None

In [95]:
df['Цена_2'] = df['Цена'].apply(extract_price)
df['Цена_2'].describe()

# вышло вроде неплохо (во всяком случае относительно похоже на правду)

count      19,737.00
mean       86,563.85
std       128,148.32
min         5,000.00
25%        39,990.00
50%        50,000.00
75%        75,000.00
max     3,000,000.00
Name: Цена_2, dtype: float64

In [96]:
# создаём 10 различных ценовых групп по процентилям
df['Ценовая_группа'] = pd.qcut(df['Цена_2'], 10, labels=False)

In [97]:
# заполняем пропуск модой по категории
df['Ремонт'] = df.groupby('Ценовая_группа')['Ремонт'].transform(lambda x: x.fillna(x.mode()[0]))

In [98]:
# двигаемся к площади комнат, я заметил, что в фичи "Площадь" иногда указано 2 или 3 числа
# посмотрев циан я узнал, что это общая площадь / жилая (площадь комнат) / площадь кухни
# соответственно идея смотреть площадь и если чисел несколько - тащить второе

In [99]:
df['Площадь комнат, м2'].isnull().sum()

7228

In [100]:
# функция вытаскивания жилой площади
def extract_room_square(text: str):
    lst = text.split('/')
    if len(lst) == 3:
        return float(lst[1])

In [101]:
# сделал такую версию, чтобы пандас не ругался
df.fillna({'Площадь комнат, м2': df['Площадь, м2'].apply(extract_room_square)}, inplace=True)

In [102]:
# прокатило для половины вариантов!
df['Площадь комнат, м2'].isnull().sum()

3692

In [103]:
# остальные заметим путём умножения площади на коэффициент площади жилья
# его можно было бы получить из нашего датасета, но данные пока не очищены
# поэтому я нагуглю - получил 0.7 что думаю недалеко от правды

SQUARE_COEF = 0.7

def extract_total_square(text: str):
    lst = text.split('/')
    return float(lst[0])

df.fillna({'Площадь комнат, м2': df['Площадь, м2'].apply(extract_total_square) * SQUARE_COEF}, inplace=True)

In [104]:
# успех
df['Площадь комнат, м2'].isnull().sum()

0

In [105]:
# настало время окон
df['Окна'].isnull().sum()

5150

In [106]:
# душа требует какой-нибудь сложной модели, но я так пока не умею
# и ничего адекватного в голову не приходит, поэтому я опять заполню модой на основе группировке по ценам
# хоть это и не звучит как самый логичный метод

In [107]:
df['Окна'] = df.groupby('Ценовая_группа')['Окна'].transform(lambda x: x.fillna(x.mode()[0]))

In [108]:
# с балконами я считаю метод простой. не указал, что он есть, значит его нет, нищеброд!
df.fillna({'Балкон': 'Нет'}, inplace=True)

In [109]:
# с санузлом мы пойдём похожим путём, только сделаем группы по площади
df['Площадь, м2_new'] = df['Площадь, м2'].apply(extract_total_square)
df['Размерная_группа'] = pd.qcut(df['Площадь, м2_new'], 10, labels=False)
df['Санузел'] = df.groupby('Размерная_группа')['Санузел'].transform(lambda x: x.fillna(x.mode()[0]))

In [110]:
# пропуск с Можно с детьми/животными я решил сделать отдельной категориальной переменной "Ни с кем нельзя!"
# тк такого варианта не было в оригинальных штуках, а точно есть люди, которые никого не одобряют
df.fillna({'Можно с детьми/животными': 'Ни с кем нельзя!'}, inplace=True)

In [111]:
# касательно "Дополнительно". Думаю врядли есть какой-то способ узнать, есть ли там холодильник, 
# кроме как звонки или просмотр фото, поэтому пропуски там оставлю пропусками
# в будущем это превратится в 0 во всех 11 категориях

In [112]:
# высоту потолков заполним медианным значением для ценовой категории
df['Высота потолков, м'] = df.groupby('Ценовая_группа')['Высота потолков, м'].transform(lambda x: x.fillna(x.median()))

In [113]:
# очистим название жк от года
def zk_separator(text):
    if isinstance(text, str):
        return text.split(',')[0]

In [114]:
df['ЖК'] =  df['Название ЖК'].apply(zk_separator)

In [115]:
# наличие лифта сначала попробуем притянуть из данных по тому же ЖК
df['Лифт'].isnull().sum()

4192

In [116]:
# убираем предупреждения
pd.set_option('future.no_silent_downcasting', True)

In [117]:
def fill_mode(group):
    mode_value = group.mode().iloc[0] if not group.mode().empty else np.nan
    return group.fillna(mode_value)

In [118]:
df.loc[df['ЖК'].notnull(), 'Лифт'] = df.groupby('ЖК')['Лифт'].transform(fill_mode)

In [119]:
# 800 лифтов заполнено, успех!
df['Лифт'].isnull().sum()

3363

In [120]:
# попробуем по адресу
df.loc[df['Адрес'].notnull(), 'Лифт'] = df.groupby('Адрес')['Лифт'].transform(fill_mode)

In [121]:
# еще 800!
df['Лифт'].isnull().sum()

2572

In [122]:
# остальные посчитаем в качестве моды для данной этажности дома
def get_floor(text):
    return int(text.split(',')[0].split('/')[1])

In [123]:
df['Этажность дома'] = df['Дом'].apply(get_floor)

In [124]:
df['Лифт'] = df.groupby('Этажность дома')['Лифт'].transform(fill_mode)

In [125]:
# осталось 5 одноэтажных домов cо спокойной душой тыкнем, что лифта нет
df[df['Лифт'].isnull()]['Этажность дома']

2578     1
11759    1
17465    1
22914    1
22954    1
Name: Этажность дома, dtype: int64

In [126]:
df.fillna({'Лифт': 'Нет'}, inplace=True)

In [127]:
df['Лифт'].isnull().sum()

0

In [128]:
# с мусоропроводом попробуем поступить также
df['Мусоропровод'].isnull().sum()

8007

In [129]:
# -2500
df.loc[df['ЖК'].notnull(), 'Мусоропровод'] = df.groupby('ЖК')['Мусоропровод'].transform(fill_mode)
df['Мусоропровод'].isnull().sum()

5633

In [130]:
# -2500
df.loc[df['ЖК'].notnull(), 'Мусоропровод'] = df.groupby('Адрес')['Мусоропровод'].transform(fill_mode)
df['Мусоропровод'].isnull().sum()

5609

In [131]:
# уже не очень логично, но раскидаем так же по этажности
df['Мусоропровод'] = df.groupby('Этажность дома')['Мусоропровод'].transform(fill_mode)
df['Мусоропровод'].isnull().sum()

1

In [132]:
# чертов один мусоропровод в 116 этажном доме 
# проверил - их там 16 https://domclick.ru/building/yuzao--tyoplyj-stan--leninskij-prospekt--123k1?utm_referrer=https%3A%2F%2Fwww.google.com%2F
df.loc[df['Мусоропровод'].isnull(), 'Этажность дома'] = 16

In [133]:
# перезапустим
df['Мусоропровод'] = df.groupby('Этажность дома')['Мусоропровод'].transform(fill_mode)
df['Мусоропровод'].isnull().sum()

0

In [134]:
# на этом моя работа по очистке всё, закинем нужные нам колонки в новый чистый датасет для 
# объединения с первой половиной фичей и дальнейшей работой по фич инженерингу

# удаляю все вспомогательные фичи (чтобы почти все из них вернуть на фазе 3 ахах)
# так же оставляю ссылку чтобы можно было что-то проверить при необходимости

df_cleared = df[['ID  объявления', 'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел', 
                 'Можно с детьми/животными', 'Дополнительно', 'Высота потолков, м', 'Лифт', 'Мусоропровод', 'Ссылка на объявление']]

In [135]:
# убеждаемся, что все чистенько, кроме "Дополнительно", но так и планировалось
df_cleared.isnull().sum()

ID  объявления                0
Ремонт                        0
Площадь комнат, м2            0
Балкон                        0
Окна                          0
Санузел                       0
Можно с детьми/животными      0
Дополнительно               272
Высота потолков, м            0
Лифт                          0
Мусоропровод                  0
Ссылка на объявление          0
dtype: int64

In [136]:
# экспорт
df_cleared.to_csv('../../data/interim/data_nun_cleared(2_of_2).csv')